# COMPOSABLE STATISTICS

Some sample scalar data:

In [195]:
(def zs [-0.178654, 0.828305, 0.0592247, -0.0121089, -1.48014, 
         -0.315044, -0.324796, -0.676357, 0.16301, -0.858164])

#'user/zs

TODO: generate new random data.

TODO: incanter integration

## RUNNING COUNT

The traditional and obvious way:

In [196]:
(reduce (fn [c z] (inc c)) 
        0 zs)

10

... with all intermediate results:

In [197]:
(reductions (fn [c z] (inc c)) 0 zs)

(0 1 2 3 4 5 6 7 8 9 10)

### THREAD-SAFE

A thread-safe way, overkill for now, but can be called on multiple threads reading from asynchronous streams. It also shows the _let-over-lambda_ (LOL) style: closing over stateful variables and using thread-safe atomic updates. This is sematically equivalent to an OOP style with mutual exclusion and transactional updates, but shorter and easier to verify.

In [198]:
(let [initial-count 0]
    (reduce ; Let-over-lambda follows
        (let [running-count (atom initial-count)]
            (fn [c z] ; closes over the atom
                (swap! running-count inc)
                @running-count)) ; ~~> new value for c
        initial-count
        zs))

10

Showing all intermediate results:

In [200]:
(let [initial-count 0]
    (reductions ; <-- this is the only difference to above
        (let [running-count (atom initial-count)]
            (fn [c z]
                (swap! running-count inc)
                @running-count)) ; ~~> new value for c
        initial-count
        zs))

(0 1 2 3 4 5 6 7 8 9 10)

### AVOIDING _REDUCE_

A thread-safe way, without `reduce`. This reduces input coupling to the environment by absorbing `initial-count` into the LOL. The environment only provides data. In this case, the environment provides data through mapping over a sequence. Below, the environment provides data through  asynchronous streams, but the LOL stays the same.

The output is still coupled to the environment through `println`. We get rid of that, too, below. 

In [240]:
(dorun 
    (map 
        (let [running-count (atom 0)]
            (fn [z]
                (swap! running-count inc)
                (print (str @running-count " "))))
        zs))

1 2 3 4 5 6 7 8 9 10 

## RUNNING MEAN

Here is a beautiful expression for the running mean, $x$. The new running mean is the old running mean plus a correction. The correction depends on new inputs $z$ but only on old statistics. That's very nice because everything except $z$ on the right-hand side of equations is prior information. The correction is a _gain_ times the _residual_. The residual is the difference betwween the new measurement $z$ and the old mean $x$. The gain is $1/(n+1)$. $n$ is a statistic, so it is an _old_ value, too: the count of observations / measurements / $z$ values seen heretofore, that is, not including the current measurement $z$. 

We can write the recurrence elegantly without subscripts as follows: $x\leftarrow{x+K\,(z-x)}$. 

In [241]:
(dorun 
    (map
        (let [running-stats (atom {:count 0, :mean 0})]
            (fn [z]
                (let [{x :mean, n :count} @running-stats
                      n+1 (inc n)
                      K   (/ 1.0 n+1)]
                    (swap! running-stats conj
                           [:count n+1]
                           [:mean (+ x (* K (- z x)))]))
                (println @running-stats)))
        zs))

{:count 1, :mean -0.178654}
{:count 2, :mean 0.3248255}
{:count 3, :mean 0.2362919}
{:count 4, :mean 0.1741917}
{:count 5, :mean -0.15667464000000003}
{:count 6, :mean -0.18306953333333337}
{:count 7, :mean -0.20331617142857145}
{:count 8, :mean -0.262446275}
{:count 9, :mean -0.21517335555555556}
{:count 10, :mean -0.27947242}


### REMOVING OUTPUT COUPLING

We can remove `println` from inside the function and lift it into a parameter. Now the function is completely decoupled from its environment.

In [242]:
(defn make-running-stats-mapper []
    (let [running-stats (atom {:count 0 :mean 0})]
        (fn [z]
            (let [{x :mean, n :count} @running-stats
                  n+1 (inc n)
                  K   (/ 1.0 n+1)]
                (swap! running-stats conj
                       [:count n+1]
                       [:mean (+ x (* K (- z x)))]))
            @running-stats)))
(clojure.pprint/pprint (map (make-running-stats-mapper) zs))

({:count 1, :mean -0.178654}
 {:count 2, :mean 0.3248255}
 {:count 3, :mean 0.2362919}
 {:count 4, :mean 0.1741917}
 {:count 5, :mean -0.15667464000000003}
 {:count 6, :mean -0.18306953333333337}
 {:count 7, :mean -0.20331617142857145}
 {:count 8, :mean -0.262446275}
 {:count 9, :mean -0.21517335555555556}
 {:count 10, :mean -0.27947242})


### NUMERICAL CHECK

In [222]:
(defn mean [zs] (/ (reduce + zs) (count zs)))
(mean zs)

-0.27947242

## CORE.ASYNC

In [223]:
(require 
    '[clojure.core.async 
      :refer 
      [sliding-buffer dropping-buffer buffer
       <!! <! >! >!! go chan onto-chan close! 
       thread alts! alts!! timeout]])

We can write on the asynch thread and read on the UI thread:

In [151]:
(let [c (chan)]
    (go (>! c 42))
    (println (<!! c))
    (close! c))

42


We can read on the async thread and write on the UI thread:

In [152]:
(let [c (chan)]
    (go (println (<! c)))
    (>!! c 42)
    (close! c))

42


In all cases, we must do any blocking call without timeout on the UI thread last. The following will hang:

In [153]:
#_(let [c (chan)]
    (>!! c 42)
    (go (println (<! c)))
    (close! c))

We're won't block if we add a timeout, but we don't know how to write to a timeout channel:

In [153]:
(let [c (chan)]
    (>!! (alts!! [c (timeout 500)]) 42)
    (go (println (<! c)))
    (close! c))

IllegalArgumentException No implementation of method: :put! of protocol: #'clojure.core.async.impl.protocols/WritePort found for class: clojure.lang.PersistentVector  clojure.core/-cache-protocol-fn (core_deftype.clj:568)


class java.lang.IllegalArgumentException: 

We can't read from a `timeout` channel either, but at least we won't hang. Here, we block-read too early on the UI thread:

In [206]:
(let [c (chan)]
    (println (<!! (alts!! [c (timeout 500)])))
    (go (>! c 42))
    (close! c))

IllegalArgumentException No implementation of method: :take! of protocol: #'clojure.core.async.impl.protocols/ReadPort found for class: clojure.lang.PersistentVector  clojure.core/-cache-protocol-fn (core_deftype.clj:568)


class java.lang.IllegalArgumentException: 

The following illustrates writing to a blocking channel at random times and reads some data on the UI thread. It will leave values in the channel and thus leak the channel according to the documentation for `close!` here https://clojure.github.io/core.async/api-index.html#C. (we have

In [207]:
(def echo-chan (chan))

(doseq   [z zs] (go (Thread/sleep (rand 100)) (>! echo-chan z)))
(dotimes [_ 3] (println (<!! echo-chan)))

(close! echo-chan)

-0.178654
-0.676357
0.0592247


We can chain channels. Reading from `echo-chan` may hang the UI thread because the UI thread races the internal thread that reads `echo-chan`, but the timeout trick works here as above. This will also leak channels.

In [207]:
(def echo-chan (chan))
(def repl-chan (chan))

(dotimes [_ 10] (go (>! repl-chan (<! echo-chan))))
(doseq   [z zs] (go (Thread/sleep (rand 100)) (>! echo-chan z)))
(dotimes [_ 3] (println (<!! (alts!! [echo-chan (timeout 500)]))))

(close! echo-chan)
(close! repl-chan)

IllegalArgumentException No implementation of method: :take! of protocol: #'clojure.core.async.impl.protocols/ReadPort found for class: clojure.lang.PersistentVector  clojure.core/-cache-protocol-fn (core_deftype.clj:568)


class java.lang.IllegalArgumentException: 

`println` on a `go` process works if we wait long enough. This, of course, is bad practice or "code smell."

In [208]:
(def echo-chan (chan))

(doseq   [z zs] (go (Thread/sleep (rand 100)) (>! echo-chan z)))
(dotimes [_ 3]  (go (println (<! echo-chan))))

(Thread/sleep 500) ; no visible output if you remove this line.
(close! echo-chan)

-0.324796
-1.48014
-0.676357


### ASYNC RUNNING MEAN

We want our `running-stats` function called with the data delivered at random times and in random order. A _transducer_, `(map mapper)`, lets us collect items off the buffer. The size of the buffer does not matter.

In [243]:
(defn async-scan [zs mapper effector]
    (let [transducer (map mapper)
          echo-chan (chan (buffer 1) transducer)]
        (doseq [z zs] (go (Thread/sleep (rand 100)) (>! echo-chan z)))
        (dotimes [_ (count zs)] (effector (<!! echo-chan)))
        (close! echo-chan)))
(async-scan zs (make-running-stats-mapper) println)

{:count 1, :mean 0.0592247}
{:count 2, :mean -0.30856615}
{:count 3, :mean -0.20974706666666665}
{:count 4, :mean -0.23607129999999998}
{:count 5, :mean -0.023196039999999973}
{:count 6, :mean 0.00783830000000002}
{:count 7, :mean -0.20473002857142852}
{:count 8, :mean -0.20147052499999996}
{:count 9, :mean -0.27443646666666666}
{:count 10, :mean -0.27947242}


We now have complete parity between space (vector `zs`) and time (values on `echo-chan` in random order).

## RUNNING STDDEV

### BRUTE-FORCE (SCALAR VERSION)

The definition of variance is the following, for $N>1$:

$$\frac{1}{N-1}\sum\limits_{i=1}^{N}\left({z_i-\bar{z}_N}\right)^2\tag{1}$$

#### SSR: SUM OF SQUARED RESIDUALS

In [210]:
(defn ssr [sequ]
    (let [m (mean sequ)]
        (reduce #(+ %1 (* (- %2 m) (- %2 m)))
                0 sequ)))
(ssr zs)

3.5566483654807355

#### VARIANCE

In [211]:
(defn variance [sequ]
    (let [n (count sequ)]
        (case n
            0 0
            1 (first sequ)
            #_default (/ (ssr sequ) (- n 1.0)))))
(variance zs)

0.3951831517200817

Let's do a smaller example:

In [212]:
(def z2s [55. 89. 144.])
(variance z2s)

2017.0

##### REALLY DUMB VARIANCE FORMULA

Here is a really dumb recurrent form. It's dumb because
1. it requires the whole sequence up front, so it cannot function in constant memory
3. the intermediate values are meaningless because they refer to the final mean and count, not to the intermediate ones

But, the final value is correct.

In [214]:
(reductions 
    (let [m (mean z2s) ; uh-oh, we refer to _all_ the data ??
          c (count z2s)]
        (fn [var z] (+ var (let [r (- z m)] ; residual
                               (/ (* r r) (- c 1.0))))))
    0 z2s)

(0 840.5 865.0 2017.0)

This is sufficiently dumb that we won't bother with a thread-safe, stateful, or asynchronous form.

##### SCHOOL VARIANCE

$$\frac{1}{N-1}\sum\limits_{i=1}^{N}\left({z_i-\bar{z}_N}\right)^2 =
\frac{1}{N-1}\left(\sum\limits_{i=1}^{N}\left(z_i^2\right)-N\,{\bar{z}_N^2}\right)\tag{2}$$

Instead of the sum of squared residuals, $ssr$, accumulate the sum of squares, $ssq$, which grows quickly. The final result is exposed to _catastrophic cancellation_, but works for our small example. We get a clue that something is not optimal with this form by the fact that we don't use the old variance to compute the new variance. We do better below.

In [245]:
(defn make-school-stats-mapper []
    (let [running-stats (atom {:count 0, :mean 0, 
                               :variance 0, :ssq 0})]
        (fn [z]
            (let [{x :mean, n :count, s :ssq} @running-stats
                  n+1 (inc n)
                  K   (/ 1.0 n+1)
                  r   (- z x)
                  x2  (+ x (* K r))
                  s2  (+ s (* z z))]
                (swap! running-stats conj
                       [:count    n+1]
                       [:mean     x2 ]
                       [:ssq      s2]
                       [:variance (/ (- s2 (* n+1 x2 x2)) (max 1 n))]))
            @running-stats)))
(clojure.pprint/pprint (map (make-school-stats-mapper) z2s))

({:count 1, :mean 55.0, :variance 0.0, :ssq 3025.0}
 {:count 2, :mean 72.0, :variance 578.0, :ssq 10946.0}
 {:count 3, :mean 96.0, :variance 2017.0, :ssq 31682.0})


##### RECURRENT VARIANCE

We already know the recurrence for the mean:

$$x\leftarrow{x+K\cdot(z-x)=x+\frac{1}{n+1}(z-x)}\tag{3}$$

The recurrence for the variance takes a little work to prove:

$$v\leftarrow\frac{\left(n-1\right)v+K\,n\,\left(z-x\right)^2}{\max(1,n)}\tag{4}$$

In [246]:
(defn make-recurrent-stats-mapper []
    (let [running-stats (atom {:count 0, :mean 0, 
                               :variance 0, :ssq 0})]
        (fn [z]
            (let [{x :mean, n :count, v :variance} @running-stats
                  n+1 (inc n)
                  K   (/ 1.0 (inc n))
                  r   (- z x)
                  x2  (+ x (* K r))
                  ssr (+ (* (- n 1) v) ; old ssr is (* (- n 1) v)
                         (* K n r r))]
                (swap! running-stats conj
                       [:count    n+1]
                       [:mean     x2 ]
                       [:variance (/ ssr  (max 1 n))]))
            @running-stats)))
(clojure.pprint/pprint (map (make-recurrent-stats-mapper) z2s))

({:count 1, :mean 55.0, :variance 0.0, :ssq 0}
 {:count 2, :mean 72.0, :variance 578.0, :ssq 0}
 {:count 3, :mean 96.0, :variance 2017.0, :ssq 0})


Of course, this works asynchronously, with potentially different intermediate values because the order is random.

In [247]:
(async-scan z2s (make-recurrent-stats-mapper) println)

{:count 1, :mean 144.0, :variance 0.0, :ssq 0}
{:count 2, :mean 116.5, :variance 1512.5, :ssq 0}
{:count 3, :mean 96.0, :variance 2017.0, :ssq 0}


#### WELFORD'S VARIANCE

The above is equivalent, algebraically and numerically, to Welford's famous recurrence for the sum of squared residuals $S$. Remember that, in recurrences, we want everything on the right-hand sides of equations or left arrows to be be old, _prior_ statistics except for the new observation / measurement / input $z$. Welford's requires the new, _posterior_ mean on the right-hand side, so it's not as elegant as our recurrence above. However, it is easier to remember!

$$S\leftarrow{S} + \left(z-x_N\right)\left(z-x_{N+1}\right)=S+\left(z-x\right)\left(z-\left(x+K\,\left(z-x\right)\right)\right)\tag{5}$$

In [248]:
(defn make-welfords-stats-mapper []
    (let [running-stats (atom {:count 0, :mean 0, :variance 0})]
        (fn [z]
            (let [{x :mean, n :count, v :variance} @running-stats
                  n+1 (inc n)
                  K   (/ 1.0 n+1)
                  r   (- z x)
                  x2  (+ x (* K r))
                  ssr (+ (* (- n 1) v) 
                         (* (- z x) (- z x2)))] ; <-- only difference to recurrent variance
                (swap! running-stats conj
                       [:count    n+1]
                       [:mean     x2 ]
                       [:variance (/ ssr  (max 1 n))]))
            @running-stats)))
(clojure.pprint/pprint (map (make-welfords-stats-mapper) z2s))

({:count 1, :mean 55.0, :variance 0.0}
 {:count 2, :mean 72.0, :variance 578.0}
 {:count 3, :mean 96.0, :variance 2017.0})


## WINDOWED STATISTICS

Suppose we want running statistics looking part way back. For example, suppose we have $N=10$ ten data and we want the statitics in a window of length $w=3$ three behind the current value, inclusively. When the first datum arrives, the window and the total include one datum. The window overhangs the left until the third datum. When the fourth datum arrives, the window contains three data and the total contain four data. After the tenth datum, we may consider three more steps marching the window "off the cliff" to the right. The following figure illustrates (the first row corresponds to $n=0$, not to $n=1$):

<img src="sliding-window.png" style="width: 200px;"/>

We won't go through the derivation of the following formulas, but rather say that they have been vetted independently (in a C program and in a Mathematica program). The following table shows a unit test that we must reproduce. The notation is explained after the table.

<img src="sliding-window-unit-test-001.png" style="width: 800px;"/>

We use the language of recurrences and avoid indices other than for datum $z_j$. The best algorithm we have found for tracking that datum (first datum to the left of the window) is to keep a FIFO queue of length $w$. This is still constant memory because it depends only on the length $w$ of the window, not on the length of the data stream.

variable | description 
---------|-------------
$n$      | prior count of data points; equals $0$ when considering the first point
$z$      | current data point
$w$      | fixed, constant, maximum width of window; $w\geq{1}$
$j$      | posterior number of points left of the window; $j\geq{0}$
$u$      | posterior number of points including $z$ in the running window; $1\leq{u}\leq{w}$
$m$      | prior mean of all points, not including $z$
$m'$     | posterior mean of all points including $z$
$m_j$    | prior mean of points left of the window, lagging $w$ behind $m$
$m'_j$   | posterior mean of points left of the window
$m'_w$   | posterior mean of points in the window
$v$      | prior variance
$v'$     | posterior variance of points including $z$
$v_j$    | prior variance of points left of the window, lagging $w$ behind $u_n$
$v'_j$   | posterior variance of points left of the window
$v'_w$   | posterior variance of points within the window

$$\begin{align}
j     &= \max(0,n+1-w)               \tag{6}\\
u     &= n-j+1                       \tag{7}\\
m'    &= m+\frac{z-m}{n+1}           \tag{8}\\
m'_j  &= \begin{cases} 
  m_j+\frac{z_j-m_j}{j} & j>0 \\
  0 & \mathrm{otherwise}
\end{cases}                            \tag{9}\\
m'_w  &= \frac{(n+1)\,m'-j\,m'_j}{u} \tag{10}\\
v'    &= \frac{(n-1)\,v+\frac{n}{n+1}\left(z-m\right)^2}{\max(1,n)}               \tag{11}\\
v'_j  &= \begin{cases}
  \frac{j-2}{j-1}\,v_j+\frac{1}{j}\,\left(z_j-m_j\right)^2 & j>1 \\
  0 & \mathrm{otherwise}
\end{cases}                                                                         \tag{12}\\
v'_w  &= \frac{n\,v'+(n-w)\,v'_j+(n+1)\,{m'}^2-j\,{m'_j}^2-u\,{m'_w}^2}{\max(1,u-1)}    \tag{13}
\end{align}$$

In [229]:
(def z3s [0.857454, 0.312454, 0.705325, 0.839363, 1.63781, 0.699257, -0.340016, -0.213596, -0.0418609, 0.054705])

#'user/z3s

In [274]:
(defn push-to-back [item vek]
    (conj (vec (drop 1 vek)) item))

#'user/push-to-back

In [280]:
(defn make-sliding-stats-mapper [w]
    (let [running-stats (atom {:n 0, :m 0, :v 0,
                               :win (vec (repeat w 0))
                               :mj 0, :vj 0})]
        (fn [z]
            (let [{:keys [m n v win mj vj]} @running-stats
                  zj   (first win)
                  win' (push-to-back z win)
                  n+1  (double (inc n))
                  n-1  (double (dec n))
                  K    (/ 1.0 n+1)
                  Kv   (* n K)
                  r    (- z m)
                  j    (max 0, (- n+1 w))
                  u    (- n+1 j)
                  m'   (+ m (* K r))
                  rj   (- zj mj)
                  mj'  (if (> j 0), (+ mj (/ rj j)), 0)
                  mw'  (/ (- (* n+1 m') (* j mj')) u)
                  v'   (/  (+ (* n-1 v) (* Kv r r))
                           (max 1 n))
                  vj'  (if (> j 1)
                           (let [j21 (/ (- j 2.0) 
                                        (- j 1.0))]
                               (+ (* j21 vj) 
                                  (/ (* rj rj) j)))
                           0)
                  vw'  (let [t1 (- (* n v')
                                   (* (- n w) vj'))
                             t2 (- (* n+1 m' m')
                                   (* j mj' mj'))
                             t3 (- (* u mw' mw'))]
                           (/  (+ t1 t2 t3)
                               (max 1 (- u 1))))
                  ]
                (swap! running-stats conj
                       [:n    n+1 ]
                       [:m    m'  ]
                       [:v    v'  ]
                       [:mj   mj' ]
                       [:vj   vj' ]
                       [:win  win']))
            @running-stats)))
(clojure.pprint/pprint (map (make-sliding-stats-mapper 3) z3s))

({:n 1.0, :m 0.857454, :v 0.0, :win [0 0 0.857454], :mj 0, :vj 0}
 {:n 2.0,
  :m 0.584954,
  :v 0.14851250000000002,
  :win [0 0.857454 0.312454],
  :mj 0,
  :vj 0}
 {:n 3.0,
  :m 0.6250776666666666,
  :v 0.07908597588033334,
  :win [0.857454 0.312454 0.705325],
  :mj 0,
  :vj 0}
 {:n 4.0,
  :m 0.678649,
  :v 0.06420353494066668,
  :win [0.312454 0.705325 0.839363],
  :mj 0.857454,
  :vj 0}
 {:n 5.0,
  :m 0.8704812,
  :v 0.23215061598970005,
  :win [0.705325 0.839363 1.63781],
  :mj 0.584954,
  :vj 0.14851250000000002}
 {:n 6.0,
  :m 0.8419438333333333,
  :v 0.1906067805693667,
  :win [0.839363 1.63781 0.699257],
  :mj 0.6250776666666666,
  :vj 0.07908597588033334}
 {:n 7.0,
  :m 0.6730924285714286,
  :v 0.35841456203828576,
  :win [1.63781 0.699257 -0.340016],
  :mj 0.678649,
  :vj 0.06420353494066666}
 {:n 8.0,
  :m 0.562256375,
  :v 0.4054895279174108,
  :win [0.699257 -0.340016 -0.213596],
  :mj 0.8704812,
  :vj 0.2321506159897}
 {:n 9.0,
  :m 0.49513223333333334,
  :v 0.3953541904

This passes the unit test.